In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,IntegerType

In [2]:
spark = SparkSession.builder.master("local[*]").appName("Demo").getOrCreate()
spark

23/04/04 21:14:08 WARN Utils: Your hostname, ZSCHN01LP0253L resolves to a loopback address: 127.0.1.1; using 192.168.1.5 instead (on interface wlp0s20f3)
23/04/04 21:14:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/04 21:14:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/04 21:14:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/04 21:14:09 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/04/04 21:14:09 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/04/04 21:14:09 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
23/04/04 21:14:09 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.


In [18]:
empDf = spark.read.option("header",True).option("infrechema",True).csv("../../Spark-main/employees.csv")
empDf.show(1)

+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|SH_CLERK|  2600|            - |       124|           50|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
only showing top 1 row



In [19]:
deptDf = spark.read.option("header",True).option("infrechema",True).csv("../../Spark-main/departments.csv")
deptDf.show(1)

+-------------+---------------+----------+-----------+
|DEPARTMENT_ID|DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-------------+---------------+----------+-----------+
|           10| Administration|       200|       1700|
+-------------+---------------+----------+-----------+
only showing top 1 row



In [20]:
from pyspark.sql.functions import *

In [21]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 104857600)
#The Value is in Bytes

In [22]:
resultDf=empDf.join(broadcast(deptDf), empDf.DEPARTMENT_ID == deptDf.DEPARTMENT_ID, "inner").select(empDf.EMPLOYEE_ID, empDf.DEPARTMENT_ID, deptDf.DEPARTMENT_NAME)
resultDf.show(1)

+-----------+-------------+---------------+
|EMPLOYEE_ID|DEPARTMENT_ID|DEPARTMENT_NAME|
+-----------+-------------+---------------+
|        198|           50|       Shipping|
+-----------+-------------+---------------+
only showing top 1 row



In [28]:
resultDf.write.option("header",True).csv("/tmp/result1")

In [29]:
resultDf.write.mode("overwrite").option("header",True).csv("/tmp/result1")

In [30]:
resultDf.write.mode("overwrite").option("header",True).option("delimiter","|").csv("/tmp/result1")

In [31]:
#JSON
resultDf.write.mode("overwrite").option("header",True).json("/tmp/result1")

In [35]:
#Default is Parquet
resultDf.write.mode("overwrite").option("header",True).save("/tmp/result1")

In [33]:
#Append
resultDf.write.mode("append").option("header",True).format("csv").save("/tmp/result1")

In [36]:
#Patition By
resultDf.write.mode("overwrite").partitionBy("DEPARTMENT_NAME").option("header",True).format("csv").save("/tmp/result1")